In [2]:
import pandas as pd

In [23]:
import numpy as np

In [8]:
from gensim.models import Word2Vec
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings


In [9]:
import os
import zipfile
import requests

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from gensim.models import FastText

In [1]:
drive_path = "/content/drive/MyDrive/Year 4/NLP notebooks and data/"

In [5]:
# Replace 'your_file.csv' with the actual path to your file in Google Drive
df = pd.read_csv('/content/drive/MyDrive/Year 4/NLP notebooks and data/dataset_cleaned.csv')

print(df.head())


   helpfulVotes           id  \
0           0.0  978474125.0   
1           0.0  978053018.0   
2           0.0  976992067.0   
3           0.0  976690540.0   
4           0.0  976664122.0   

                                             address     city  \
0  24 Dharmapala Mawatha Dharmapala Mawatha, Colo...  Colombo   
1  24 Dharmapala Mawatha Dharmapala Mawatha, Colo...  Colombo   
2  24 Dharmapala Mawatha Dharmapala Mawatha, Colo...  Colombo   
3  24 Dharmapala Mawatha Dharmapala Mawatha, Colo...  Colombo   
4  24 Dharmapala Mawatha Dharmapala Mawatha, Colo...  Colombo   

               placeName  numberOfReviews  placeRating  \
0  Nh Collection Colombo             2392          4.5   
1  Nh Collection Colombo             2392          4.5   
2  Nh Collection Colombo             2392          4.5   
3  Nh Collection Colombo             2392          4.5   
4  Nh Collection Colombo             2392          4.5   

                                              webUrl publishedDate 

# GloVe

In [11]:
text = df['review_text'].astype(str).tolist()

In [13]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
word_index = tokenizer.word_index

In [14]:
# Pad sequences - for training
# max_sequence_length = max(len(seq) for seq in sequences)
# data = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

In [15]:
# Step 3: Download and Load GloVe Embeddings
glove_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
glove_zip = 'glove.6B.zip'
glove_dir = 'glove.6B'

In [16]:
# Download GloVe embeddings
if not os.path.exists(glove_zip):
    print("Downloading GloVe embeddings...")
    response = requests.get(glove_url)
    with open(glove_zip, 'wb') as f:
        f.write(response.content)

In [17]:
# Extract GloVe embeddings
if not os.path.exists(glove_dir):
    with zipfile.ZipFile(glove_zip, 'r') as zip_ref:
        zip_ref.extractall(glove_dir)

In [18]:
# Load GloVe embeddings
embedding_index = {}
embedding_dim = 100  # Using 100-dimensional GloVe vectors
glove_file = os.path.join(glove_dir, f'glove.6B.{embedding_dim}d.txt')

In [24]:
with open(glove_file, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [25]:
# Prepare embedding matrix
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [26]:
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [32]:
# Save the embedding matrix to a file
np.save(drive_path+'glove_embedding_matrix.npy', embedding_matrix)

In [33]:
# Load the embedding matrix from the file
embedding_matrix = np.load(drive_path+'glove_embedding_matrix.npy')

In [34]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.024843    0.47766     0.32437    ... -0.22634     0.82833999
   0.058472  ]
 [ 0.43044001 -0.71715999  0.13989    ...  0.082734    1.28840005
  -0.019587  ]
 ...
 [-0.10628     0.39139     0.43858999 ... -0.94142997 -0.074166
   1.27040005]
 [ 0.12223     0.14296     0.15203001 ...  0.29115999 -0.12074
   0.10729   ]
 [-0.17752001  0.60913998  0.35699001 ... -0.050707    0.18080001
   0.40375999]]
